In [85]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [86]:
df = pd.read_csv('raw_dataset.csv')
df.drop(columns=df.columns[0], inplace=True)
df = df[len(df) - 377:]
df.reset_index(inplace=True)
df.drop(columns=df.columns[0], inplace=True)

pca = pd.read_csv('predictions_pca.csv')
kpca = pd.read_csv('predictions_kpca.csv')
df

,Date,Close,Volume,SPYt,SPYt1,SPYt2,SPYt3,RDP5,RDP10,RDP15,...,DJI,IXIC,AAPL,MSFT,XOM,GE,JNJ,WFC,AMZN,JPM
0,2011-11-28,119.709999,210686000.0,0.021465,-0.001887,-0.020397,-0.003928,-1.860964,-5.487134,-4.598346,...,11523.009766,2527.340088,13.432857,24.870001,75.839996,70.928070,62.369999,24.150000,9.7075,29.160000
1,2011-11-29,120.050003,199241500.0,0.002840,0.021520,-0.001887,-0.020347,0.325923,-4.312128,-4.918421,...,11555.629883,2515.510010,13.328571,24.840000,76.930000,71.503166,62.779999,24.080000,9.4195,28.559999
2,2011-11-30,124.989998,324439500.0,0.021465,0.002840,0.021523,-0.001887,4.866176,-0.864534,-2.259931,...,12045.679688,2620.340088,13.650000,25.580000,80.440002,76.247681,64.720001,25.860001,9.6145,30.969999
3,2011-12-01,124.970001,176954800.0,-0.000160,0.021520,0.002840,0.021440,5.155793,0.717279,1.469631,...,12020.030273,2626.199951,13.854643,25.280001,79.790001,76.247681,64.449997,25.639999,9.8565,30.459999
4,2011-12-02,124.860001,221109700.0,-0.000880,-0.000160,0.021523,0.002840,5.155793,2.252068,0.434364,...,12019.419922,2626.929932,13.917857,25.219999,79.790001,77.110313,63.470001,26.070000,9.8015,32.330002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,2013-05-23,165.449997,211064400.0,-0.002893,-0.007418,0.001438,-0.000060,0.066530,1.577844,3.568073,...,15294.500000,3459.419922,15.790714,34.114999,91.790001,113.389069,75.744995,40.009998,13.0900,53.349998
373,2013-05-24,165.309998,151573900.0,-0.000846,-0.002893,-0.007418,0.001438,-0.976402,1.162716,2.441595,...,15303.099609,3459.139893,15.898214,34.114999,91.529999,112.766052,75.744995,40.240002,13.0870,53.660000
374,2013-05-28,166.300003,143679800.0,0.005989,-0.000846,-0.002893,-0.007418,-0.377398,1.687667,2.793920,...,15409.389648,3472.009949,15.765714,34.114999,92.379997,113.101524,75.744995,40.520000,13.3645,54.599998
375,2013-05-29,165.220001,160363400.0,-0.006494,0.005989,-0.000846,-0.002893,-1.166475,-0.006049,1.611313,...,15302.799805,3467.520020,15.891071,34.114999,92.080002,113.293221,75.744995,40.750000,13.2765,54.669998


# Backtesting the strategy on the testing data

In [88]:
from tqdm import tqdm
import numpy as np

# PCA

pnls_pca = []
pnls_kpca = []

for j in range(len(pca)):

    df_pca = pca.loc[j]
    df_kcpa = kpca.loc[j]

    pnl = [0]
    quantity = 1
    in_position = ''
    df['pnl_pca'] = 0

    for i in tqdm(range(1, len(df))):

        signal = df_pca[i]

        # Entries and exits
        if signal == 1:
            df['pnl_pca'][i] = df['pnl_pca'][i-1] + df['SPYt'][i]
        else:
            T1H = df['T1'][i]/100 * 7/90        # get the daily value of the tbill from the monthly one
            df['pnl_pca'][i] = df['pnl_pca'][i-1] + T1H


    # KPCA

    pnl = [0]
    quantity = 1
    in_position = ''
    df['pnl_kpca'] = 0

    for i in tqdm(range(1, len(df))):

        signal = df_kcpa[i]

        # Entries and exits
        if signal == 1:
            df['pnl_kpca'][i] = df['pnl_kpca'][i-1] + df['SPYt'][i]
        else:
            T1H = df['T1'][i] * 7/90        # get the daily value of the tbill from the monthly one
            df['pnl_kpca'][i] = df['pnl_kpca'][i-1] + T1H

    pnls_pca.append(df['pnl_pca'])
    pnls_kpca.append(df['pnl_kpca'])

100%|██████████| 376/376 [00:00<00:00, 73354.96it/s]


In [89]:
import plotly.graph_objects as go

n_pc = [1, 3, 6, 10, 15, 22, 26, 31, 34, 37, 40, 60]    # number of principal components for the transformed datasets

traces1 = []
traces2 = []

for i in range(len(pnls_pca)):
    trace1 = go.Scatter(x=df.index, y=pnls_pca[i], mode='lines', name='PCA ' + str(n_pc[i]))
    trace2 = go.Scatter(x=df.index, y=pnls_kpca[i], mode='lines', name='KPCA ' + str(n_pc[i]))
    traces1.append(trace1)
    traces2.append(trace2)

# Create a figure with the traces
fig = go.Figure(traces1)
fig.update_layout(title='PnL')
fig.show()

In [92]:
fig = go.Figure(traces2)
fig.update_layout(title='PnL')
fig.show()


# Comparing to a buy and hold strategy

In [96]:
df['bnh_Tbill'] = 0
df['bnh_Tbill'] = df['bnh_Tbill'].shift(1) + df['T1'] * 7/90
df['bnh_SPY'] = 0
df['bnh_SPY'] = df['bnh_SPY'].shift(1) + df['SPYt']


import plotly.graph_objects as go

# Create traces for each line
trace1 = go.Scatter(x=df.index, y=df['bnh_Tbill'], mode='lines', name='PnL Tbills')
trace2 = go.Scatter(x=df.index, y=df['bnh_SPY'], mode='lines', name='PnL SPY')

trace3 = go.Scatter(x=df.index, y=pnls_pca[1], mode='lines', name='PnL PCA')
trace4 = go.Scatter(x=df.index, y=pnls_kpca[-4], mode='lines', name='PnL KPCA')


fig = go.Figure([trace1, trace2, trace3, trace4])

fig.update_layout(title='PnL')

fig.show()

# Stats about the strategies

In [98]:
pnl1 =pnls_pca[1]
pnl2 = pnls_kpca[-4]

rf_r = 1 - df['T60'][376]   # Risk free rate

# Sharpe Ratio
sharpe_ratio_1 = np.mean(pnl1) - rf_r / np.std(pnl1)
sharpe_ratio_2 = np.mean(pnl2) - rf_r/ np.std(pnl2)

# Maximum Drawdown

def calculate_maximum_drawdown(pnl):
    peak = pnl[0]
    max_drawdown = 0
    for i in range(1, len(pnl)):
        if pnl[i] > peak:
            peak = pnl[i]
        else:
            if peak != 0:
                drawdown = (peak - pnl[i]) / peak
            if drawdown > max_drawdown:
                max_drawdown = drawdown
    return max_drawdown


max_drawdown_1 = calculate_maximum_drawdown(pnl1)
max_drawdown_2 = calculate_maximum_drawdown(pnl2)

# Win-Loss Ratio
win_loss_ratio_1 = np.sum(np.diff(pnl1) > 0) / np.sum(np.diff(pnl1) < 0)
win_loss_ratio_2 = np.sum(np.diff(pnl2) > 0) / np.sum(np.diff(pnl2) < 0)


print("Strategy PCA 3:")
print("Sharpe Ratio:", sharpe_ratio_1)
print("Maximum Drawdown:", max_drawdown_1)
print("Win-Loss Ratio:", win_loss_ratio_1)

print("\nStrategy KPCA 34:")
print("Sharpe Ratio:", sharpe_ratio_2)
print("Maximum Drawdown:", max_drawdown_2)
print("Win-Loss Ratio:", win_loss_ratio_2)


Strategy PCA 3:
Sharpe Ratio: 0.22840448019380943
Maximum Drawdown: 1.5427538889915382
Win-Loss Ratio: 3.032967032967033

Strategy KPCA 34:
Sharpe Ratio: 0.37053030931729664
Maximum Drawdown: 1.1704146496710206
Win-Loss Ratio: 2.0081967213114753
